# このノートブックについて

predictした値をベイズ推定してみる

# インポート

In [12]:
import numpy as np
import glob
import cv2
from keras.models import load_model

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

# 関数

In [9]:
def process_img(img_list):

    image_array = np.empty((0, 224,224,3))
    for img_path in img_list:
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.reshape(1, 224, 224, 3)

        image_array = np.concatenate([image_array, img], axis=0)

    image_array /= 255
    return image_array

def get_labels(num_0, num_1, num_2):
    t = np.concatenate([np.zeros(num_0), np.ones(num_1), np.ones(num_2) * 2]).astype(int)
    t = np.identity(3)[t].astype(int)
    return t

# モデルのロード

In [10]:
model = load_model("../model/img_rec/VGG19_pred.h5")

# データのロード

In [15]:
path1 = "../dataset/data_reshape/image_onehand_reshape/N"
path2 =  "../dataset/data_reshape/image_onehand_reshape/H"
path3 = "../dataset/data_reshape/image_onehand_reshape/R"
N_imgtri_list = glob.glob(path1 + "/*" + ".png")
print(len(N_imgtri_list))
H_imgtri_list = glob.glob(path2 + "/*" + ".png")
print(len(H_imgtri_list))
R_imgtri_list = glob.glob(path3 + "/*" + ".png")
print(len(R_imgtri_list))
N_arr = process_img(N_imgtri_list)
H_arr = process_img(H_imgtri_list)
R_arr = process_img(R_imgtri_list)
x = np.concatenate([N_arr, H_arr, R_arr], axis=0)
print(x.shape)
t = get_labels(27, 40, 49)
print(t.shape)
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size=0.2)

27
40
49
(116, 224, 224, 3)
(116, 3)


# 推定

In [16]:
y_pred = model.predict(x_test)

# ベイス推定1

Q,陽性と予測された人の中で、本当に病気である人の確率は?

前提：へバーデン結節と関節リウマチを合わせて「罹患」として、まとめる。

In [26]:
# 予測値の混同行列
conf = confusion_matrix(np.argmax(t_test, axis=1), np.argmax(y_pred, axis=1))
conf

array([[ 5,  0,  0],
       [ 0,  8,  1],
       [ 0,  0, 10]])

## へバーデン結節と関節リウマチをまとめた行列を作成

In [19]:
a = np.sum(conf[1:, 0], axis=0)
b = np.sum(conf[0, 1:], axis=0)
c = np.sum(conf[1:, :], axis=0)
c = np.sum(c[1:])

In [20]:
bays_mat = np.empty((2,2))
bays_mat[0, 0] = conf[0, 0]
bays_mat[1, 0] = a
bays_mat[0, 1] = b
bays_mat[1, 1] = c
bays_mat = bays_mat.astype(int)
bays_mat

array([[ 5,  0],
       [ 0, 19]])

## ベイズ推定に必要な確率を算出

In [27]:
# P(陽性|罹患)
a_ = bays_mat[0, 0] / np.sum(bays_mat[0])

# 事前分布
pre_prob = (H_arr.shape[0] + R_arr.shape[0]) / (N_arr.shape[0] + H_arr.shape[0] + R_arr.shape[0])
pre_prob = round(pre_prob, 2)

# P(陽性|非罹患)
b_ = bays_mat[1, 0] / np.sum(bays_mat[1])

# P(陽性)
pre_plus = pre_prob * a_ + (1 - pre_prob) * b_

## 結果

In [32]:
result = (a_ * pre_prob) / pre_plus
print("result:", result, sep="")

result_per = round(result, 2)
result_per = result_per*100
print("result(%):", result_per.astype(int), sep="")

result:1.0
result(%):100


## 結論

A, 陽性と予測された人の中で、本当に病気である人の確率は**100%**

# ベイズ推定２

Q,陰性と予測された人の中で、実は病気である人の確率は?

前提：上記と同じ

## ベイズ推定に必要な確率を算出

In [33]:
# P(陰性|罹患)
a_ = bays_mat[0, 1] / np.sum(bays_mat[0])

# 事前分布
pre_prob = (H_arr.shape[0] + R_arr.shape[0]) / (N_arr.shape[0] + H_arr.shape[0] + R_arr.shape[0])
pre_prob = round(pre_prob, 2)

# P(陰性|非罹患)
b_ = bays_mat[1, 1] / np.sum(bays_mat[1])

# P(陰性)
pre_plus = pre_prob * a_ + (1 - pre_prob) * b_

## 結果

In [34]:
result = (a_ * pre_prob) / pre_plus
print("result:", result, sep="")

result_per = round(result, 2)
result_per = result_per*100
print("result(%):", result_per.astype(int), sep="")

result:0.0
result(%):0


# 考察

非常に高い値が出たが、実際は非現実的である。  
データが少ないことと、それに伴う事前分布が高すぎる事によるものと考えられる。  
実際に分析するときはもっと慎重になる必要がある。